In [1]:
!export LC_ALL=en_US.UTF-8
import tensorflow as tf
import json
from pprint import pprint
import re
import numpy as np
from embedding import *

In [2]:
data = json.load(open('../train-v1.1.json'))

In [3]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [4]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [5]:
X_train_comp_all = []
X_train_comp_ans_all = []
X_train_ans_all = []
Y_train_ques_all = []
invalid = 0
X_train_ans_label_all = []
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
        start,end = find_sub_list(answer,passage)
        if start == -1:
            continue
        a_lab[start:end+1] = 1
            
            
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_ans_all.append(findAnsVec(answer,passage))
            X_train_ans_label_all.append(a_lab)
            X_train_comp_all.append(cappedPassage)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1
    

In [6]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all for item in sublist] + [item for sublist in Y_train_ques_all for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [7]:
wordToTake = 50000
words = findKMostFrequentWords(100000)

In [8]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
        if(len(reduced_glove) == wordToTake):
            break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)
reduced_glove.shape

(50000, 300)

In [9]:
!hostname
print(invalid)
for i in np.where(X_train_ans_label_all[110] == 1)[0]:
    print(X_train_comp_all[110][i])

cuda2.cims.nyu.edu
1510
rev
.
john
j.
cavanaugh
,
c.s.c
.
served
more
than
half
,
lobund
institute
for
animal
studies
and
medieval
institute
.
hall
of
liberal
arts
(


In [10]:
print(X_train_comp_all[0])
print(X_train_ans_all[0])

['architecturally', ',', 'the', 'school', 'has', 'a', 'catholic', 'character', '.', 'atop', 'the', 'main', 'building', "'s", 'gold', 'dome', 'is', 'a', 'golden', 'statue', 'of', 'the', 'virgin', 'mary', '.', 'immediately', 'in', 'front', 'of', 'the', 'main', 'building', 'and', 'facing', 'it', ',', 'is', 'a', 'copper', 'statue', 'of', 'christ', 'with', 'arms', 'upraised', 'with', 'the', 'legend', '``', 'ad', 'me', 'omnes', "''", '.', 'next', 'to', 'the', 'main', 'building', 'is', 'the', 'basilica', 'of', 'the', 'sacred', 'heart', '.', 'immediately', 'behind', 'the', 'basilica', 'is', 'the', 'grotto', ',', 'a', 'marian', 'place', 'of', 'prayer', 'and', 'reflection', '.', 'it', 'is', 'a', 'replica', 'of', 'the', 'grotto', 'at', 'lourdes', ',', 'france', 'where', 'the', 'virgin', 'mary', 'reputedly', 'appeared', 'to', 'saint', 'bernadette', 'soubirous', 'in', '1858.', 'at', 'the', 'end', 'of', 'the', 'main', 'drive', '(', 'and', 'in', 'a', 'direct', 'line', 'that', 'connects', 'through', '

In [11]:
find_sub_list(X_train_ans_all[0] , X_train_comp_all[0])

(101, 104)

In [12]:
print(invalid)
print(X_train_comp_all[101])
print(X_train_ans_all[101])
print(Y_train_ques_all[101])

c = list(zip(X_train_comp_all,X_train_comp_ans_all, X_train_ans_all, X_train_ans_label_all,Y_train_ques_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_comp_ans_all_shuffled, X_train_ans_shuffled, X_train_ans_label_shuffled,Y_train_ques_all_shuffled = zip(*c)

print(X_train_comp_all_shuffled[101])
print(X_train_comp_ans_all_shuffled[101])
print(X_train_ans_shuffled[101])
print(X_train_ans_label_shuffled[101])
print(Y_train_ques_all_shuffled[101])


1510
['the', 'success', 'of', 'its', 'football', 'team', 'made', 'notre', 'dame', 'a', 'household', 'name', '.', 'the', 'success', 'of', 'note', 'dame', 'reflected', 'rising', 'status', 'of', 'irish', 'americans', 'and', 'catholics', 'in', 'the', '1920s', '.', 'catholics', 'rallied', 'up', 'around', 'the', 'team', 'and', 'listen', 'to', 'the', 'games', 'on', 'the', 'radio', ',', 'especially', 'when', 'it', 'knocked', 'off', 'the', 'schools', 'that', 'symbolized', 'the', 'protestant', 'establishment', 'in', 'america', '—', 'harvard', ',', 'yale', ',', 'princeton', ',', 'and', 'army', '.', 'yet', 'this', 'role', 'as', 'high-profile', 'flagship', 'institution', 'of', 'catholicism', 'made', 'it', 'an', 'easy', 'target', 'of', 'anti-catholicism', '.', 'the', 'most', 'remarkable', 'episode', 'of', 'violence', 'was', 'the', 'clash', 'between', 'notre', 'dame', 'students', 'and', 'the', 'ku', 'klux', 'klan', 'in', '1924.', 'nativism', 'and', 'anti-catholicism', ',', 'especially', 'when', 'dire

In [13]:
examples_to_take_train = 1000

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_comp_ans = X_train_comp_ans_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_shuffled[0:examples_to_take_train]
X_train_ans_label = X_train_ans_label_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
answer_indices = [np.where(x==1)[0].tolist() for x in X_train_comp_ans]

In [14]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [15]:
document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
suppression_answer = np.zeros((examples_to_take_train, reduced_glove.shape[0], 1),dtype=np.int32)
print(suppression_answer.shape)

(1000, 50000, 1)


In [16]:
print(answer_labels.shape)
for i in range(examples_to_take_train):
    answer_labels[i,0:len(X_train_ans_label[i])] = X_train_ans_label[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    for j, index in enumerate(answer_indices[i]):
        answer_masks[i, j, index] = 1
    answer_lengths[i] = len(answer_indices[i])
    
    #print(Y_train_ques[i])
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)
    
    for j, word in enumerate(X_train_ans[i]):
        if(word not in Y_train_ques[i]):
            suppression_answer[i, look_up_word_reduced(word),:] = 1

(1000, 449)


In [17]:
def create_vocabulary(data):
    flat_list = [item for sublist in data for item in sublist]
    vocabulary = sorted(set(flat_list))
    vocabulary.append("<UNK>")
    vocabulary.append("unk")
    vocabulary.append("eos")
    vocabulary = ["<EOS>"] + vocabulary
    word_to_index = { word:i for i,word in enumerate(vocabulary) }
    index_to_word = { i:word for i,word in enumerate(vocabulary) }
    return (vocabulary,word_to_index,index_to_word)

In [18]:
print(reduced_glove.shape)
vocabulary_comp,word_to_index_comp,index_to_word_comp = create_vocabulary(X_train_comp + Y_train_ques)
print(len(vocabulary_comp))
print(word_to_index_comp["?"])
print(word_to_index_comp["what"])

(50000, 300)
18128
1439
17650


In [19]:
def create_one_hot_vector(data,vocabulary,word_to_index,index_to_word, maxLen):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,word in enumerate(data):
        if i >= maxLen:
            break
        if(word not in word_to_index):
            word = "<UNK>"
        one_hot[i][word_to_index[word]] = 1
    return one_hot

def create_one_hot_vector_from_indices(data,maxLen,vocabulary):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,indice in enumerate(data):
        if i >= maxLen:
            break
        one_hot[i][int(indice)] = 1
    return one_hot


def create_one_hot_training_Set(data,maxLen,vocabulary):
    one_hot_data = np.zeros([data.shape[0],maxLen,len(vocabulary)])
    for i in range(data.shape[0]):
        one_hot_data[i] = create_one_hot_vector_from_indices(data[i],maxLen,vocabulary)
    return one_hot_data




In [20]:
def sentences_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(END_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
    return X_indices

In [21]:
document_tokens = sentences_to_indices_glove(X_train_comp, max_document_len)

In [22]:
document_tokens[0]

array([   22,  1795,   385, 19053,     4,  2629,  4553,  7562, 30963,
          35,  3788, 16025,   139,     3,  2592,     5, 24499,    31,
           3,  4237,     5,     3,    65,  4329,   104,     4,     3,
        2493,   167,     8, 19252,     6,     3, 16025,  6960,     7,
        5149,     3,   167,     7,     3,  5646,     4,    51,  2760,
         482,     3,  2592,     5, 24499,     7,  7018,  1734,    91,
       28738,     4,   601,  4932,     4,     7,     3,   700,     5,
           3,   911,     6,    48,    85,   110,  2355,   464,    22,
           3,   545,  9146,     9,  2582,   765,     4,     3, 23165,
           5,     7,  9146,     9,     3,  4329,     4,     3, 20042,
           4,  5304,  8759, 14065,     4, 15324,     4,   338,    22,
           3,   700,     5,     3,   911,     4,     3,  5357,   157,
           4, 22835,     7, 13628,     4,     3,   393,     5,  2276,
        2427,     8,   277,  2642,     4,     3,   855,    73,  1376,
           7,     3,

In [23]:
answer_labels.shape

(1000, 449)

## Model

### Document Embedding

In [40]:
import tensorflow.contrib.seq2seq as seq2seq

tf.reset_default_graph()

embedding = tf.get_variable("embedding", initializer=reduced_glove)
embedding = tf.cast(embedding, dtype=tf.float32)
EMBEDDING_DIMENS = glove.shape[1]

d_tokens = tf.placeholder(tf.int32, shape=[None, None], name="d_tokens")
d_lengths = tf.placeholder(tf.int32, shape=[None], name="d_lengths")


document_emb = tf.nn.embedding_lookup(embedding, d_tokens, name="document_emb")
document_emb = tf.cast(document_emb, dtype=tf.float64, name="casted_document_emb")


### Answer Generation

In [41]:

forward_cell = tf.contrib.rnn.GRUCell(EMBEDDING_DIMENS)
backward_cell = tf.contrib.rnn.GRUCell(EMBEDDING_DIMENS)

answer_outputs, _ = tf.nn.bidirectional_dynamic_rnn(
    forward_cell, backward_cell, document_emb, d_lengths, dtype=tf.float64,
    scope="answer_rnn")

answer_outputs = tf.concat(answer_outputs, 2, name="answer_output_concat")

answer_outputs = tf.cast(answer_outputs,tf.float32, name="answer_output_concat")

answer_tags = tf.layers.dense(inputs=answer_outputs, units=2, name="answer_tags")


a_labels = tf.placeholder(tf.int32, shape=[None, None], name="a_labels")

answer_mask = tf.sequence_mask(d_lengths, dtype=tf.float32, name="answer_mask")

answer_loss = seq2seq.sequence_loss(
    logits=answer_tags, targets=a_labels, weights=answer_mask, name="answer_loss")

## Question Generation

### Encoder

In [42]:
encoder_input_mask = tf.placeholder(
    tf.float32, shape=[None, None, None], name="encoder_input_mask")
encoder_inputs = tf.matmul(encoder_input_mask, answer_outputs, name="encoder_inputs")
encoder_lengths = tf.placeholder(tf.int32, shape=[None], name="encoder_lengths")

encoder_cell = tf.contrib.rnn.GRUCell(forward_cell.state_size + backward_cell.state_size)

_, encoder_state = tf.nn.dynamic_rnn(
    encoder_cell, encoder_inputs, encoder_lengths, dtype=tf.float32, scope="encoder_rnn")

### Decoder

In [43]:
from tensorflow.python.layers.core import Dense


decoder_inputs = tf.placeholder(tf.int32, shape=[None, None], name="decoder_inputs")
decoder_lengths = tf.placeholder(tf.int32, shape=[None], name="decoder_lengths")

decoder_emb = tf.nn.embedding_lookup(embedding, decoder_inputs,name="decoder_embedding")
decoder_emb = tf.cast(decoder_emb,tf.float32,name="decoder_embedding_cast")

#helper = seq2seq.TrainingHelper(decoder_emb , decoder_lengths2, name="helper")
helper = seq2seq.TrainingHelper(decoder_emb , decoder_lengths, name="helper")


projection = Dense(embedding.shape[0], use_bias=False, name="projection")

decoder_cell = tf.contrib.rnn.GRUCell(encoder_cell.state_size)


decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)
decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, scope="decoder")
decoder_outputs = decoder_outputs.rnn_output

decoder_labels = tf.placeholder(tf.int64, shape=[None, None], name="decoder_labels")
question_mask = tf.sequence_mask(decoder_lengths ,dtype=tf.float32)


## Question Generation Loss

In [44]:
# NLL Loss
question_loss = seq2seq.sequence_loss(
    logits=decoder_outputs, targets=decoder_labels, weights=question_mask,
    name="question_loss")


#Suppression Loss
s_answer = tf.placeholder(tf.float32, shape=[None,None,None], name="suppression_answer")
lambdaSuppress = 0.5
decoder_outputs = tf.Print(decoder_outputs, [decoder_outputs], message="This is decoder_outputs: ")
suppression_loss = lambdaSuppress * tf.reduce_sum(tf.matmul(tf.exp(decoder_outputs), s_answer))
suppression_loss = tf.Print(suppression_loss, [suppression_loss], message="This is suppression_loss: ")

In [45]:
loss = tf.add(answer_loss, question_loss, name="loss")
loss = tf.add(loss, suppression_loss, name="loss")

In [30]:
#document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
#document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
#answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
#answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
#question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
#question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#suppression_answer = np.zeros((examples_to_take_train,max_answer_len),dtype=np.int32)

In [31]:
def shuffle_list(*ls):
    l =list(zip(*ls))
    np.random.shuffle(l)
    return zip(*l)

In [46]:
import math
def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    num_batches = math.ceil(len(inputs[0])/batch_size)
    
    for index,inp in enumerate(inputs):
        start = 0
        output = []
        for i in range(num_batches-1):
            maxD = max(inputs[1][start:start+batch_size])
            maxA = max(inputs[4][start:start+batch_size])
            maxQ = max(inputs[7][start:start+batch_size])
            if index == 0 or index == 2:
                output.append(inp[start:start+batch_size,0:maxD]) 
            elif index==3:
                output.append(inp[start:start+batch_size,0:maxA,0:maxD]) 
            elif index==5 or index==6:
                output.append(inp[start:start+batch_size,0:maxQ])
            elif index == 8: # 
                output.append(inp[start:start+batch_size,:,:]) 
            else: 
                output.append(inp[start:start+batch_size])
            start = start + batch_size
        
        # Remaining training sample i.e. training mod batch_size
        maxD = max(inputs[1][start:])
        maxA = max(inputs[4][start:])
        maxQ = max(inputs[7][start:])
        if index == 0 or index == 2:
            output.append(inp[start:,0:maxD]) 
        elif index==3:
            output.append(inp[start:,0:maxA,0:maxD]) 
        elif index==5 or index==6:
            output.append(inp[start:,0:maxQ]) 
        elif index ==8:
            output.append(inp[start:,:, :]) 
        else: 
            output.append(inp[start:])
        outputs.append(output)
    
    return outputs


In [47]:
batch_size = 10
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_masks,answer_lengths,question_input_tokens,question_output_tokens,question_lengths,suppression_answer]
                    ,batch_size)

In [48]:
"No of batches:",len( batch_input[0])

('No of batches:', 100)

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=3e-3).minimize(loss)

saver = tf.train.Saver()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.InteractiveSession(config=config)

session.run(tf.global_variables_initializer())
# session.run(tf.variables_initializer(saved_vars))


EPOCHS = 20

for epoch in range(1, EPOCHS + 1):
    print("Epoch {0}".format(epoch))
    for batchNum in range(len(batch_input[0])):
        print("Batch : ",batchNum)
        t = session.run([optimizer, loss, decoder_outputs, suppression_loss], {
            d_tokens: batch_input[0][batchNum],
            d_lengths: batch_input[1][batchNum],
            a_labels: batch_input[2][batchNum],
            encoder_input_mask: batch_input[3][batchNum],
            encoder_lengths: batch_input[4][batchNum],
            decoder_inputs: batch_input[5][batchNum],
            decoder_labels: batch_input[6][batchNum],
            decoder_lengths: batch_input[7][batchNum],
            s_answer: batch_input[8][batchNum],
        })
        print("Loss: {0}".format(t[1]))
    if(epoch%5 == 0):
        print("Saving model")
        #saver.save(session, "qgen-model")


Epoch 1
Batch :  0
Loss: 232.2726593017578
Batch :  1
Loss: 274.4121398925781
Batch :  2
Loss: 145.31298828125
Batch :  3
Loss: 73.01394653320312
Batch :  4
Loss: 479.9945373535156
Batch :  5
Loss: 21.947721481323242
Batch :  6
Loss: 36.79529571533203
Batch :  7
Loss: 30.75307846069336
Batch :  8
Loss: 12.670621871948242
Batch :  9
Loss: 111.17853546142578
Batch :  10
Loss: 83.9248046875
Batch :  11
Loss: 26.95000457763672
Batch :  12
Loss: 32.0759391784668
Batch :  13
Loss: 13.022895812988281
Batch :  14
Loss: 13.940422058105469
Batch :  15
Loss: 23.330528259277344
Batch :  16
Loss: 9.2161865234375
Batch :  17
Loss: 12.02365493774414
Batch :  18
Loss: 23.711360931396484
Batch :  19
Loss: 10.296273231506348
Batch :  20
Loss: 11.635177612304688
Batch :  21
Loss: 9.511051177978516
Batch :  22
Loss: 9.635671615600586
Batch :  23
Loss: 11.360706329345703
Batch :  24
Loss: 11.241825103759766
Batch :  25
Loss: 10.623144149780273
Batch :  26
Loss: 9.375879287719727
Batch :  27
Loss: 9.6872320

Loss: 7.423376083374023
Batch :  29
Loss: 7.185211658477783
Batch :  30
Loss: 6.504748344421387
Batch :  31
Loss: 7.012627601623535
Batch :  32
Loss: 7.206829071044922
Batch :  33
Loss: 6.905035018920898
Batch :  34
Loss: 6.394003868103027
Batch :  35
Loss: 6.274222373962402
Batch :  36
Loss: 6.606213569641113
Batch :  37
Loss: 6.856496810913086
Batch :  38
Loss: 6.688869953155518
Batch :  39
Loss: 6.5604705810546875
Batch :  40
Loss: 6.983190536499023
Batch :  41
Loss: 6.574142932891846
Batch :  42
Loss: 6.862995624542236
Batch :  43
Loss: 6.281550407409668
Batch :  44
Loss: 5.911989212036133
Batch :  45
Loss: 6.7705817222595215
Batch :  46
Loss: 7.308733940124512
Batch :  47
Loss: 6.354945182800293
Batch :  48
Loss: 5.946995258331299
Batch :  49
Loss: 6.197289943695068
Batch :  50
Loss: 6.007554531097412
Batch :  51
Loss: 6.88509464263916
Batch :  52
Loss: 6.438209056854248
Batch :  53
Loss: 6.752099514007568
Batch :  54
Loss: 6.8063225746154785
Batch :  55
Loss: 6.3842291831970215
B

Loss: 5.873495101928711
Batch :  57
Loss: 5.597692966461182
Batch :  58
Loss: 5.599781036376953
Batch :  59
Loss: 5.553738117218018
Batch :  60
Loss: 5.633862495422363
Batch :  61
Loss: 5.625927448272705
Batch :  62
Loss: 5.440574645996094
Batch :  63
Loss: 6.284501552581787
Batch :  64
Loss: 5.662989616394043
Batch :  65
Loss: 5.711974620819092
Batch :  66
Loss: 5.616353988647461
Batch :  67
Loss: 5.230132102966309
Batch :  68
Loss: 5.094156742095947
Batch :  69
Loss: 5.764741897583008
Batch :  70
Loss: 5.6909050941467285
Batch :  71
Loss: 4.718756198883057
Batch :  72
Loss: 5.493353366851807
Batch :  73
Loss: 5.365789890289307
Batch :  74
Loss: 5.458958625793457
Batch :  75
Loss: 5.940831184387207
Batch :  76
Loss: 5.541176795959473
Batch :  77
Loss: 5.817055702209473
Batch :  78
Loss: 5.306770324707031
Batch :  79
Loss: 5.824134349822998
Batch :  80
Loss: 5.983602046966553
Batch :  81
Loss: 5.190749645233154
Batch :  82
Loss: 5.402558326721191
Batch :  83
Loss: 5.52564001083374
Batc

Loss: 4.225068092346191
Batch :  85
Loss: 3.885964870452881
Batch :  86
Loss: 4.398972034454346
Batch :  87
Loss: 4.881364822387695
Batch :  88
Loss: 4.773468494415283
Batch :  89
Loss: 4.432913780212402
Batch :  90
Loss: 4.490651607513428
Batch :  91
Loss: 4.623348236083984
Batch :  92
Loss: 8.954522132873535
Batch :  93
Loss: 4.313284397125244
Batch :  94
Loss: 4.738758087158203
Batch :  95
Loss: 5.049170017242432
Batch :  96
Loss: 5.5059967041015625
Batch :  97
Loss: 5.430203914642334
Batch :  98
Loss: 5.878780364990234
Batch :  99
Loss: 5.942455291748047
Epoch 8
Batch :  0
Loss: 5.498902320861816
Batch :  1
Loss: 5.6360650062561035
Batch :  2
Loss: 5.261592388153076
Batch :  3
Loss: 5.426651954650879
Batch :  4
Loss: 6.171736717224121
Batch :  5
Loss: 5.332431793212891
Batch :  6
Loss: 5.427584648132324
Batch :  7
Loss: 5.349945545196533
Batch :  8
Loss: 5.402608394622803
Batch :  9
Loss: 5.4499406814575195
Batch :  10
Loss: 5.6967902183532715
Batch :  11
Loss: 5.305965900421143
Ba

Loss: 3.909174680709839
Batch :  12
Loss: 4.063060283660889
Batch :  13
Loss: 3.8217849731445312
Batch :  14
Loss: 4.181344509124756
Batch :  15
Loss: 4.047612190246582
Batch :  16
Loss: 3.543832540512085
Batch :  17
Loss: 3.7757606506347656
Batch :  18
Loss: 3.181278705596924
Batch :  19
Loss: 3.601701498031616
Batch :  20
Loss: 3.8277432918548584
Batch :  21
Loss: 3.334277629852295
Batch :  22
Loss: 3.519456386566162
Batch :  23
Loss: 3.3259007930755615
Batch :  24
Loss: 3.99045729637146
Batch :  25
Loss: 3.683467388153076
Batch :  26
Loss: 3.0562965869903564
Batch :  27
Loss: 3.90948748588562
Batch :  28
Loss: 4.3975958824157715
Batch :  29
Loss: 3.7338435649871826
Batch :  30
Loss: 3.547039747238159
Batch :  31
Loss: 3.5046987533569336
Batch :  32
Loss: 4.036134243011475
Batch :  33
Loss: 3.650454521179199
Batch :  34
Loss: 3.4224140644073486
Batch :  35
Loss: 3.2632601261138916
Batch :  36
Loss: 3.5608792304992676
Batch :  37
Loss: 3.802885055541992
Batch :  38
Loss: 3.62244677543

Loss: 2.090219736099243
Batch :  37
Loss: 2.2811057567596436
Batch :  38
Loss: 2.320932388305664
Batch :  39
Loss: 2.3607563972473145
Batch :  40
Loss: 2.5861690044403076
Batch :  41
Loss: 2.4403178691864014
Batch :  42
Loss: 2.4532744884490967
Batch :  43
Loss: 1.7282763719558716
Batch :  44
Loss: 2.0423033237457275
Batch :  45
Loss: 1.9753785133361816
Batch :  46
Loss: 2.5732269287109375
Batch :  47
Loss: 1.8082501888275146
Batch :  48
Loss: 1.7415486574172974
Batch :  49
Loss: 2.0778896808624268
Batch :  50
Loss: 1.843035340309143
Batch :  51
Loss: 2.5965123176574707
Batch :  52
Loss: 2.030379056930542
Batch :  53
Loss: 2.2865946292877197
Batch :  54
Loss: 2.6772994995117188
Batch :  55
Loss: 1.9923254251480103
Batch :  56
Loss: 2.573425054550171
Batch :  57
Loss: 2.361879825592041
Batch :  58
Loss: 1.8917326927185059
Batch :  59
Loss: 2.0885701179504395
Batch :  60
Loss: 2.267735719680786
Batch :  61
Loss: 2.0327205657958984
Batch :  62
Loss: 2.081230640411377
Batch :  63
Loss: 2.0

Loss: 1.441646933555603
Batch :  61
Loss: 1.3191155195236206
Batch :  62
Loss: 1.2419353723526
Batch :  63
Loss: 1.2778549194335938
Batch :  64
Loss: 1.3462048768997192
Batch :  65
Loss: 1.7126545906066895
Batch :  66
Loss: 1.3625879287719727
Batch :  67
Loss: 0.928442120552063
Batch :  68
Loss: 1.0959209203720093
Batch :  69
Loss: 1.3328137397766113
Batch :  70
Loss: 1.190093994140625
Batch :  71
Loss: 0.9441039562225342
Batch :  72
Loss: 1.4681552648544312
Batch :  73
Loss: 1.4860478639602661
Batch :  74
Loss: 1.2522250413894653
Batch :  75
Loss: 1.3187129497528076
Batch :  76
Loss: 1.2567378282546997
Batch :  77
Loss: 1.7259953022003174
Batch :  78
Loss: 1.228967308998108
Batch :  79
Loss: 1.657352328300476
Batch :  80
Loss: 1.3634533882141113
Batch :  81
Loss: 1.219016194343567
Batch :  82
Loss: 1.4933913946151733
Batch :  83
Loss: 1.3038687705993652
Batch :  84
Loss: 1.3020403385162354
Batch :  85
Loss: 1.1719930171966553
Batch :  86
Loss: 1.2488116025924683
Batch :  87
Loss: 1.39

In [39]:
print(t[2])

[[[-1.50505602 -3.07428956  2.56434155 ..., -3.07349539 -3.09046841
   -3.03399873]
  [-1.76248872 -3.63838267  3.2439065  ..., -3.60971045 -3.58260107
   -3.6037097 ]
  [-1.89708757 -3.88434219  3.52075863 ..., -3.82470012 -3.79168701
   -3.84030294]
  ..., 
  [-2.04667497 -4.1406498   3.82709146 ..., -4.07375765 -4.03911209
   -4.09230137]
  [-2.04751468 -4.14365482  3.82962894 ..., -4.07715559 -4.04227924
   -4.09470797]
  [-2.04816628 -4.14639616  3.83192492 ..., -4.08027935 -4.04514122
   -4.09690571]]

 [[-1.8915031  -3.94711781  3.36775589 ..., -3.78215742 -3.83540106
   -3.81771779]
  [-1.97481418 -4.14009571  3.65373945 ..., -3.9985826  -4.01781034
   -4.04910374]
  [-2.0104506  -4.19613743  3.75386572 ..., -4.06355572 -4.07419825
   -4.12057781]
  ..., 
  [-2.06712484 -4.23035145  3.86246133 ..., -4.13321733 -4.09855413
   -4.1683917 ]
  [-2.06826019 -4.22900057  3.86204839 ..., -4.13337708 -4.09799767
   -4.16759014]
  [-2.06893539 -4.2278266   3.86186838 ..., -4.13366032 -4

In [ ]:
saver = tf.train.Saver()
saver.save(session, "qgen-model")

In [ ]:
batch_input[3][0].shape

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
saver = tf.train.Saver()

saver.restore(session, 'qgen-model')

In [ ]:
answers = session.run(answer_tags, {
    d_tokens: batch_input[0][48],
    d_lengths: batch_input[1][48],
})
answers = np.argmax(answers, 2)

In [ ]:
for i in range(28):
    print("Prediction")
    printAllAns(answers,48,0)
    print("Ground Truth")
    printAllAns(batch_input[2][48],48,0)

In [ ]:
def printDoc(batch,num):
    for i in batch_input[0][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")

def printQues(batch,num):
    for i in batch_input[5][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")
    
def printAnsForQuestion(batch, num):
    for i in batch_input[5][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")
    
def printAllAns(answers, batch, num):
    for i,word in enumerate(batch_input[0][batch][num]):
        if answers[num][i] == 1 :
            print(look_up_token_reduced(word),sep=" ", end=" ")
    print(" ")
    
printQues(47,0)
    

In [ ]:
import itertools

batchNum = 80

helper = seq2seq.GreedyEmbeddingHelper(embedding, tf.fill([batch_input[0][batchNum].shape[0]], START_TOKEN), END_TOKEN)
decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)
decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, maximum_iterations=max_question_len)
decoder_outputs = decoder_outputs.rnn_output


questions = session.run(decoder_outputs, {
    d_tokens: batch_input[0][batchNum],
    d_lengths: batch_input[1][batchNum],
    a_labels: batch_input[2][batchNum],
    encoder_input_mask: batch_input[3][batchNum],
    encoder_lengths: batch_input[4][batchNum],
})



In [ ]:
batch_input[0][batchNum].shape

In [ ]:
#questions[:,:,END_TOKEN] = 0
qs = np.argmax(questions, 2)

In [ ]:
# q = set()
l = []
for i in range(batch_input[0][batchNum].shape[0]):
    print("---------------------------------------------------------------------------------------------")
    question = itertools.takewhile(lambda t: t != END_TOKEN, qs[i])
    print("Generated Question: " + " ".join(look_up_token_reduced(token) for token in question))
    print("Ground Truth Question: ")
    printQues(batchNum,i)
    print("Ground Truth Answer: ", X_train_ans_shuffled[batch_size*batchNum + i])
    print("Context:")
    printDoc(batchNum,i)
    print("---------------------------------------------------------------------------------------------")
    
    
# q

In [ ]:
batch_input[5][18].shape

In [ ]:
batch_input[5][1].shape

In [ ]:
batch_input[6][1].shape

In [ ]:
max(batch_input[7][1])

In [ ]:
max_question_len

In [ ]:
len(batch_input)

In [ ]:
reduced_glove